In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import defaultdict
import networkx as nx
import time


In [87]:
x1 = pd.read_csv('comprehensive_medical_notes.csv')  
y1 = pd.read_csv('new_ground_truth.csv')

In [88]:
def evaluate_pairs(pred_pairs, ground_truth_clusters):
    true_pairs = set()
    for cluster in ground_truth_clusters:
        for a in range(len(cluster)):
            for b in range(a+1, len(cluster)):
                i,j = cluster[a], cluster[b]
                true_pairs.add((min(i,j), max(i,j)))

    all_pairs = pred_pairs.union(true_pairs)
    y_true = [1 if p in true_pairs else 0 for p in all_pairs]
    y_pred = [1 if p in pred_pairs  else 0 for p in all_pairs]

    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred,
        average='binary',
        zero_division=0
    )
    return {'precision': p, 'recall': r, 'f_score': f1}

In [89]:
class EntityResolutionPipeline:
    def __init__(self):
        self.data = None
        self.cleaned_data = None
        self.blocks = defaultdict(list)
        self.candidate_pairs = set()
        # self.matches = set()
        # self.clusters = []
        
    def set_data(self, data):
        self.data = data.copy()
        self.cleaned_data = data.copy()
        self.cleaned_data['cleaned_title'] = self.cleaned_data['important_notes'].apply(str)
        return self.data
    
    def clean_data(self):
        self.cleaned_data = self.data.copy()
        extra_words = {
            'bundle', 'warranty', 'deal', 'offers?', 'shipping',
            'windows', 'win(?:\d+)?', 'pro', 'home', 'ultimate',
            'beats', 'audio'
        } 
        common_words = [
            'vology', '-', '/', '"', 'hdd', 'ssd', 'windows ', 'windows',
            'laptop', 'new', 'sale', 'core', 'buy', 'computers', 'computer', 
            'computers', 'com'
        ]
        def clean_title(title):
            title = title.lower()
            title = re.sub(r'[!@#$%^&*()_+={}\[\]|\\:;<>?,./"]', ' ', title)
            title_tokens = title.split()
            title_tokens = [token for token in title_tokens if token not in ENGLISH_STOP_WORDS]
            title = ' '.join(title_tokens)
            title = re.sub(r'(\d+)\s*gb', r'\1gb', title)  
            title = re.sub(r'(\d+)\s*tb', r'\1tb', title) 
            title = re.sub(r'core\s*i(\d)', r'corei\1', title)  
            title = re.sub(r'windows\s*(\d+)', r'windows\1', title) 
            title = re.sub(r'gigs', 'gb', title)
            
            for i in common_words:
                title = title.replace(i, '')

            for i in extra_words:
                title = title.replace(i, '')
            
            return title
        
        self.cleaned_data['cleaned_title'] = self.cleaned_data['important_notes'].apply(clean_title)
        return self.cleaned_data
    
    def create_blocks(self, max_blocks=5000):    
        self.blocks = defaultdict(list)
        
        for idx, row in self.cleaned_data.iterrows():
            tokens = row['cleaned_title'].split()
            for token in tokens:
                if len(token) > 2:  
                    self.blocks[token].append(row['patient_id'])
        
        return self.blocks

    
    def filter_blocks(self, tau):
        max_blocks=5000
        # 1) filter by tau
        filtered_blocks = {
            token: records
            for token, records in self.blocks.items()
            if 1 < len(records) < tau
        }

        # (optional) prune to max_blocks as we discussed before
        if max_blocks and len(filtered_blocks) > max_blocks:
            tokens_sorted = sorted(
                filtered_blocks.keys(),
                key=lambda t: len(filtered_blocks[t]),
                reverse=True,
            )
            keep = set(tokens_sorted[:max_blocks])
            filtered_blocks = {t: filtered_blocks[t] for t in keep}

        # **store** the filtered blocks for later inspection
        self.filtered_blocks = filtered_blocks

        # 2) build candidate pairs
        self.candidate_pairs = set()
        for records in filtered_blocks.values():
            for i in range(len(records)):
                for j in range(i + 1, len(records)):
                    self.candidate_pairs.add((records[i], records[j]))

        # **return** both, so you can optionally unpack them
        return filtered_blocks, self.candidate_pairs

    
    def compute_jaccard_similarity(self, id1, id2):
        if not hasattr(self, '_id_to_row_index'):
            self._id_to_row_index = {row['patient_id']: i for i, row in self.cleaned_data.iterrows()}
        try:
            idx1 = self._id_to_row_index[id1]
            idx2 = self._id_to_row_index[id2]
            
            tokens1 = set(self.cleaned_data.iloc[idx1]['cleaned_title'].split())
            tokens2 = set(self.cleaned_data.iloc[idx2]['cleaned_title'].split())
            
            intersection = len(tokens1.intersection(tokens2))
            union = len(tokens1.union(tokens2))
            
            return intersection / union if union > 0 else 0
        except (KeyError, IndexError):
            return 0

    def find_matches(self, alpha):
        self.matches = set()
        if not hasattr(self, '_id_to_row_index'):
            self._id_to_row_index = {row['patient_id']: i for i, row in self.cleaned_data.iterrows()}
        
        for id1, id2 in self.candidate_pairs:
            similarity = self.compute_jaccard_similarity(id1, id2)
            if similarity > alpha:
                self.matches.add((min(id1, id2), max(id1, id2)))
        
        return self.matches
    
    # def pivot_clustering(self):
    #     adj_list = defaultdict(list)
    #     all_ids = set()
        
    #     for id1, id2 in self.matches:
    #         adj_list[id1].append(id2)
    #         adj_list[id2].append(id1)
    #         all_ids.add(id1)
    #         all_ids.add(id2)
        
    #     remaining_nodes = set(all_ids)
    #     clusters = []
        
    #     while remaining_nodes:
    #         pivot = None
    #         max_degree = -1
    #         for node in remaining_nodes:
    #             degree = len(adj_list[node])
    #             if degree > max_degree:
    #                 max_degree = degree
    #                 pivot = node

    #         if pivot is None or max_degree == 0:
    #             if remaining_nodes:
    #                 pivot = next(iter(remaining_nodes))
    #                 clusters.append([pivot])
    #                 remaining_nodes.remove(pivot)
    #             continue
            
    #         neighbors = adj_list[pivot]
    #         cluster = [pivot] + neighbors
    #         clusters.append(cluster)

    #         for node in cluster:
    #             if node in remaining_nodes:
    #                 remaining_nodes.remove(node)
        
    #     self.clusters = clusters
    #     return self.clusters

In [ ]:
data = x1
# truth_labels = y1

# edges = [(row['lid'], row['rid']) for _, row in truth_labels.iterrows()]
# ground_truth_clusters = []
# used_ids = set()

# for id1, id2 in edges:
#     found_cluster = None
#     for cluster in ground_truth_clusters:
#         if id1 in cluster or id2 in cluster:
#             found_cluster = cluster
#             break
    
#     if found_cluster:
#         found_cluster.add(id1)
#         found_cluster.add(id2)
#     else:
#         ground_truth_clusters.append({id1, id2})
    
#     used_ids.add(id1)
#     used_ids.add(id2)

# ground_truth_clusters = [list(cluster) for cluster in ground_truth_clusters]

pipeline = EntityResolutionPipeline()
pipeline.set_data(data)

In [10]:
# def plot_recall_vs_tau(data, ground_truth_clusters):
#     """Plot recall of candidate pairs as a function of tau"""
#     pipeline = EntityResolutionPipeline()
#     pipeline.set_data(data)
#     pipeline.clean_data()
#     pipeline.create_blocks()
    
#     tau_values = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#     recalls = []
    
#     for tau in tau_values:
#         candidates = pipeline.filter_blocks(tau)
#         metrics = evaluate_pairs(candidates, ground_truth_clusters)
#         recalls.append(metrics['recall'])
    
#     plt.figure(figsize=(10, 6))
#     plt.plot(tau_values, recalls, marker='o', linestyle='-')
#     plt.xlabel('τ (Block Size Threshold)')
#     plt.ylabel('Recall')
#     plt.title('Recall of Candidate Pairs vs. τ')
#     plt.grid(True)
#     plt.savefig('recall_vs_tau.png')
#     plt.show()
    
#     return pd.DataFrame({'tau': tau_values, 'recall': recalls})

In [9]:
# def plot_metrics_vs_alpha(data, ground_truth_clusters, fixed_tau=50):
#     """Plot precision, recall, and F-measure vs alpha with a fixed tau value"""
#     pipeline = EntityResolutionPipeline()
#     pipeline.set_data(data)
#     pipeline.clean_data()
#     pipeline.create_blocks()
#     pipeline.filter_blocks(fixed_tau)
    
#     alpha_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#     precisions = []
#     recalls = []
#     f_scores = []
    
#     for alpha in alpha_values:
#         matches = pipeline.find_matches(alpha)
#         metrics = evaluate_pairs(matches, ground_truth_clusters)
#         precisions.append(metrics['precision'])
#         recalls.append(metrics['recall'])
#         f_scores.append(metrics['f_score'])
    
#     plt.figure(figsize=(10, 6))
#     plt.plot(alpha_values, precisions, marker='o', linestyle='-', label='Precision')
#     plt.plot(alpha_values, recalls, marker='s', linestyle='-', label='Recall')
#     plt.plot(alpha_values, f_scores, marker='^', linestyle='-', label='F-measure')
#     plt.xlabel('α (Similarity Threshold)')
#     plt.ylabel('Score')
#     plt.title(f'Precision, Recall, and F-measure vs. α (τ={fixed_tau})')
#     plt.grid(True)
#     plt.legend()
#     plt.savefig('metrics_vs_alpha.png')
#     plt.show()
    
#     return pd.DataFrame({
#         'alpha': alpha_values,
#         'precision': precisions,
#         'recall': recalls,
#         'f_score': f_scores
#     })

In [7]:
# print("\n--- Question 2a ---")
# recall_data = plot_recall_vs_tau(data, ground_truth_clusters)
# print("Generated plot of recall vs τ")
# print("Recall values:")
# print(recall_data)

In [8]:
# print("\n--- Question 2b ---")
# metrics_data = plot_metrics_vs_alpha(data, ground_truth_clusters)
# print("Generated plot of precision, recall, and F-measure vs α")
# print("Metrics values:")
# print(metrics_data)

In [91]:
filtered_blocks_global = []
def run_pipeline(tau, alpha, clean_the_data =True):  
    blocks = defaultdict(list)
    candidate_pairs = set()
    matches = set()
    clusters = []
    tau = 50
    alpha = 0.7
    
    start_time = time.time()
    pipeline = EntityResolutionPipeline()
    data = pd.read_csv('compiled_important_notes.csv')
    pipeline.set_data(data)
    ground_truth = pd.read_csv('new_primary_diagnosis.csv').values.tolist()
        
    if clean_the_data:
        data = pipeline.clean_data()

    # Blocking
    blocking_start = time.time()
    # candidates = pipeline.filter_blocks(tau)
    # blocking_time = time.time() - blocking_start
    # print(f"Blocking completed in {blocking_time:.2f} seconds, generated {len(candidates)} candidate pairs")
    
    # print(list(candidates)[0])    

    blocks = pipeline.create_blocks()
    filtered_blocks, candidates = pipeline.filter_blocks(tau)
    filtered_blocks_global = filtered_blocks

    print(f"Kept {len(filtered_blocks)} blocks; generated {len(candidates)} candidate pairs")

    # inspect the *first* filtered block:
    first_token, first_records = next(iter(filtered_blocks.items()))
    print("First block token:", first_token)
    print("Records in that block:", first_records)
    blocking_time = time.time() - blocking_start
    

    # Evaluate blocking
    # block_metrics = evaluate_pairs(candidates, ground_truth)
    # print(f"  → Blocking metrics:  precision={block_metrics['precision']:.3f},"
    #         f" recall={block_metrics['recall']:.3f},"
    #         f" F₁={block_metrics['f_score']:.3f}\n")


   
    # Pair matching
    matching_start = time.time()
    matches = pipeline.find_matches(alpha)
    matching_time = time.time() - matching_start
    print(f"Pair matching completed in {matching_time:.2f} seconds, found {len(matches)} matches")
    print(list(matches)[0])


    
    # match_metrics = evaluate_pairs(matches, ground_truth)
    # print(f"  → Pair‐matching metrics:  precision={match_metrics['precision']:.3f},"
    #         f" recall={match_metrics['recall']:.3f},"
    #         f" F₁={match_metrics['f_score']:.3f}\n")
    
    # Clustering
    # clustering_start = time.time()
    # edges = [(row['lid'], row['rid']) for _, row in truth_labels.iterrows()]
    # ground_truth_clusters = []
    # used_ids = set()
    # for id1, id2 in edges:
    #     found_cluster = None
    #     for cluster in ground_truth_clusters:
    #         if id1 in cluster or id2 in cluster:
    #             found_cluster = cluster
    #             break
        
    #     if found_cluster:
    #         found_cluster.add(id1)
    #         found_cluster.add(id2)
    #     else:
    #         ground_truth_clusters.append({id1, id2})
        
    #     used_ids.add(id1)
    #     used_ids.add(id2)

    # ground_truth_clusters = [list(cluster) for cluster in ground_truth_clusters]
    # clusters = pipeline.pivot_clustering()
    # print(f"Created {len(clusters)} clusters")
    # print("Sample clusters:")
    # for i, cluster in enumerate(clusters[:5]):
    #     for record_id in cluster[:3]:
    #         record_title = pipeline.cleaned_data.loc[pipeline.cleaned_data['id'] == record_id, 'cleaned_title'].iloc[0]
    #     if len(cluster) > 3:
    #         print(f"  ... and {len(cluster) - 3} more records")

    # clustering_time = time.time() - clustering_start
    # print(f"Clustering completed in {clustering_time:.2f} seconds, found {len(clusters)} clusters")
    
    # # Evaluate clustering
    # cluster_pairs = set()
    # for cluster in clusters:
    #     for i in range(len(cluster)):
    #         for j in range(i+1, len(cluster)):
    #             id1, id2 = cluster[i], cluster[j]
    #             cluster_pairs.add((min(id1, id2), max(id1, id2)))

    # cluster_metrics = evaluate_pairs(cluster_pairs, ground_truth_clusters)

    # print(f"  → Clustering metrics:  precision={cluster_metrics['precision']:.3f},"
    #               f" recall={cluster_metrics['recall']:.3f},"
    #               f" F₁={cluster_metrics['f_score']:.3f}\n")
    
    total_time = time.time() - start_time
    print(f"Total execution time: {total_time:.2f} seconds")
    
    return {
        'blocking': {
            # 'precision': block_metrics['precision'],
            # 'recall': block_metrics['recall'],
            # 'f1': block_metrics['f_score'],
            'time': blocking_time,
            'candidates': len(candidate_pairs)
        },
        'matching': {
            # 'precision': match_metrics['precision'],
            # 'recall': match_metrics['recall'],
            # 'f1': match_metrics['f_score'],
            'time': matching_time,
            'matches': len(matches)
        },
        # 'clustering': {
        #     'precision': cluster_metrics['precision'],
        #     'recall': cluster_metrics['recall'],
        #     'f1': cluster_metrics['f_score'],
        #     'time': clustering_time,
        #     'clusters': len(clusters)
        # },
        'total_time': total_time, 
        'filtered_blocks_global': filtered_blocks_global

    }

In [11]:
# def compare_cleaning(tau, alpha):
#     with_cleaning = run_pipeline(tau, alpha, True)
#     without_cleaning = run_pipeline(tau, alpha, False)
    
#     print("\nComparison:")
#     print(f"With cleaning - Final F1: {with_cleaning['matching']['f1']:.4f}, Total time: {with_cleaning['total_time']:.2f}s")
#     print(f"Without cleaning - Final F1: {without_cleaning['matching']['f1']:.4f}, Total time: {without_cleaning['total_time']:.2f}s")
    
#     return with_cleaning, without_cleaning

In [ ]:
# Question 2c and 2d
print("\n--- Question 2c & 2d ---")
best_results = run_pipeline(50, 0.7, False)

import pandas as pd

# Read your main data
df = pd.read_csv('compiled_important_notes.csv')

# Initialize a list to hold result rows
block_rows = []
filtered_blocks_global = best_results['filtered_blocks_global']

# Iterate through all blocks
for token, patient_ids in filtered_blocks_global.items():
    # Filter the dataframe to just patients in this block
    matches = df[df['patient_id'].isin(patient_ids)]

    # Aggregate the first 5 notes per patient_id
    agg_dict = (
        matches
        .groupby('patient_id')['important_notes']
        .apply(lambda notes: ' '.join(notes.head(5)))
        .to_dict()
    )

    # Build the strings
    block_list_str = ','.join(map(str, patient_ids))
    all_notes_list = [agg_dict[pid] for pid in patient_ids if pid in agg_dict]
    all_notes_str = ' '.join(all_notes_list)

    # Save the row
    block_rows.append({
        'block_list_str': block_list_str,
        'all_notes': all_notes_str
    })

# Convert to DataFrame
result_df = pd.DataFrame(block_rows)

# Save to CSV
result_df.to_csv('filtered_blocks_notes.csv', index=False)

# Optional: Show preview
print(result_df.head().to_string(index=False))



In [ ]:
import pandas as pd

# 1) Load your master list of patient IDs
all_patients = pd.read_csv('compiled_important_notes.csv')['patient_id'].astype(int)

test_ids = all_patients.sample(frac=0.03, random_state=50).tolist()
# Or sample a fixed number: test_ids = all_patients.sample(n=100, random_state=42).tolist()

# 3) Load the precomputed blocks + notes CSV
blocks_df = pd.read_csv('filtered_blocks_notes.csv')
blocks_df['patient_list'] = blocks_df['block_list_str'].str.split(',')

# 4) Explode to one row per patient
exploded = (
    blocks_df
    .explode('patient_list')
    .rename(columns={'patient_list': 'patient_id'})
)
exploded['patient_id'] = exploded['patient_id'].astype(int)

# 5) Filter to just your test IDs
test_set = exploded[exploded['patient_id'].isin(test_ids)][
    ['patient_id', 'block_list_str', 'all_notes']
].drop_duplicates('patient_id').reset_index(drop=True)

# 6) Save your test set for model evaluation
test_set.to_csv('er_test_set.csv', index=False)

print(f"Saved {len(test_set)} rows to er_test_set.csv")


In [ ]:
import pandas as pd
import requests
import time

API_KEY = ''
API_URL = 'https://api.perplexity.ai/chat/completions'
MODEL   = 'llama-3.1-sonar-large-128k-online'
INPUT_CSV  = 'er_test_set.csv'
OUTPUT_CSV = 'er_test_icd9_codes.csv'

df = pd.read_csv(INPUT_CSV)

def diagnose_icd9(notes: str) -> str:
    system_msg = {
        "role": "system",
        "content": (
            "You are a board‑certified medical coder. It is crucial that you are providing the right range in IC9 codes."
            "Return exactly one ICD‑9 code for these clinical notes. Only include the code and no other words with nothing following or before it."
        )
    }
    user_msg = {
        "role": "user",
        "content": notes
    }

    payload = {
        "model": MODEL,
        "messages": [system_msg, user_msg],
        "max_tokens": 10,
        "temperature": 0.0,
        "stream": False
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    resp = requests.post(API_URL, json=payload, headers=headers)
    resp.raise_for_status()
    content = resp.json()['choices'][0]['message']['content'].strip()
    integer_code = content.split('.')[0]
    return integer_code

results = []
for _, row in df.iterrows():
    pid   = row['patient_id']
    notes = row['all_notes']
    try:
        code = diagnose_icd9(notes)
    except Exception:
        code = None
    results.append({"patient_id": pid, "icd9_code": code})

# Save only patient_id + icd9_code
out_df = pd.DataFrame(results)
out_df.to_csv(OUTPUT_CSV, index=False)

# And if you just want to print the list in Python:
print(out_df.values.tolist())


In [ ]:
import pandas as pd
import ast

# Load ER data
er_df = pd.read_csv("er_test_icd9_codes.csv")

# Load ground truth (assuming it's a CSV as well)
ground_truth_df = pd.read_csv("new_ground_truth.csv")

import ast

def safe_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

ground_truth_df['diagnosis'] = ground_truth_df['diagnosis'].apply(safe_eval)


# Merge ER data with ground truth on patient_id
merged_df = pd.merge(er_df, ground_truth_df, on='patient_id', how='left')

# Convert ICD9 code to string to match diagnosis format
merged_df['icd9_code'] = merged_df['icd9_code'].astype(str)

def is_code_in_same_range(icd9_code, diagnosis_list):
    try:
        icd9_int = int(float(icd9_code))  # handles cases like "438.1"
        icd9_range = icd9_int // 100
        for diag in diagnosis_list:
            try:
                diag_int = int(float(diag))
                if diag_int // 100 == icd9_range:
                    return True
            except (ValueError, TypeError):
                continue
        return False
    except (ValueError, TypeError):
        return False

merged_df['is_match'] = merged_df.apply(
    lambda row: is_code_in_same_range(row['icd9_code'], row['diagnosis']) if isinstance(row['diagnosis'], list) else False,
    axis=1
)


# Calculate accuracy
accuracy = merged_df['is_match'].mean()

# Print results
print(f"Accuracy: {accuracy:.2%}")
print(merged_df[['patient_id', 'icd9_code', 'diagnosis', 'is_match']])
